In [ ]:
# default_exp config

# Config

> This module manages configuration of the `planetarypy` package.

In [ ]:
# hide
from nbdev.showdoc import show_doc

In [ ]:
# export
import os
import shutil
from functools import reduce
from importlib.resources import path as resource_path
from fastcore.utils import Path

import strictyaml as yaml

In [ ]:
# exports
class Config:
    """Manage config stuff.

    Attributes
    -------
    path: pathlib.Path

    The key, value pairs found in the config file become attributes of the
    class instance after initialization.
    At minimum, there should be the `storage_root` attribute for storing data
    for this package.
    """

    # This part enables a config path location override using env PYCISS_CONFIG
    fname = "planetarypy_config.yaml"
    # separating fname from fpath so that resource_path below is correct.
    path = Path(os.getenv("PLANETARYPY_CONFIG", Path.home() / f".{fname}"))

    def __init__(self, config_path=None):
        "Switch to other config file location with `config_path`."
        if config_path is not None:
            self.path = Path(config_path)
        if not self.path.exists():
            with resource_path("planetarypy.data", self.fname) as p:
                shutil.copy(p, self.path)
        self.read_config()

    def read_config(self):
        """Read the configfile and store config dict.

        If found, load config via `yaml` and store YAML dict as `d`.
        `storage_root` will be stored as attribute.
        """
        self.yamldic = yaml.load(self.path.read_text())
        if not self.yamldic["storage_root"].data:
            self.ask_storage_root()
        else:
            self.storage_root = Path(self.yamldic["storage_root"].data)

    @property
    def d(self):
        "get the Python dic from YAML dict."
        return self.yamldic.data

    def get_value(self, key):
        """Get sub-dictionary by nested key.

        Parameters
        ----------
        nested_key: str
            A nested key in dotted format, e.g. cassini.uvis.indexes
        """
        return reduce(lambda c, k: c[k], key.split("."), self.d)

    def set_value(self, nested_key, value, save=True):
        """Set sub-dic using dotted key.

        Parameters
        ----------
        key: str
            A nested key in dotted format, e.g. cassini.uvis.ring_summary
        value: convertable to string
            Value for the given key to be stored.
        """
        dic = self.yamldic
        keys = nested_key.split(".")
        for key in keys[:-1]:
            dic = dic[key]
        dic[keys[-1]] = value
        if save:
            self.save()

    def save(self):
        "Write the YAML dict to file."
        self.path.write_text(self.yamldic.as_yaml())

    def ask_storage_root(self):
        """Use input() to ask user for the storage_root path.

        The path will be stored in the YAML-dict and saved into existing config file
        at `Class.path`, either default or as given during init.
        `storage_root` attribute is set as well.
        """
        path = input(
            "Provide the root storage path where all downloaded and produced data will be stored:"
        )
        self.yamldic["storage_root"] = path
        self.storage_root = Path(path)
        self.save()

    def list_missions(self):
        return list(self.d["missions"].keys())

    def list_instruments(self, mission):
        if not mission.startswith("missions"):
            mission = "missions." + mission
        instruments = self.get_value(mission)
        return list(instruments.keys())

    def list_indexes(self, instrument):
        "instrument key needs to be <mission>.<instrument>"
        if not instrument.startswith("missions"):
            instrument = "missions." + instrument
        indexes = self.get_value(instrument + ".indexes")
        return list(indexes)

    def get_clean_copy(self, new_path):
        "Create a clean copy without timestamps for repo upload."
        dic = self.yamldic.copy()
        missions = dic['missions']
        for mission in missions.keys():
            mdict = missions[mission]
            for instr in mdict.keys():
                instrdict = mdict[instr]
                for index in instrdict['indexes']:
                    instrdict['indexes'][index]['timestamp']=''
        Path(new_path).write_text(dic.as_yaml())
        return Config(config_path=new_path)

In [ ]:
config = Config()

In [ ]:
dic = config.yamldic.copy()

In [ ]:
clean_config = config.get_clean_copy("test_config.yaml")

In [ ]:
clean_config.path

Path('test_config.yaml')

## Reference
The nested structure code was taken from this brilliant blog post:

https://nvie.com/posts/modifying-deeply-nested-structures


In [ ]:
missions = dic['missions']
missions

YAML({'cassini': {'iss': {'indexes': {'index': {'url': 'https://pds-rings.seti.org/holdings/metadata/COISS_2xxx/COISS_2999/COISS_2999_index.lbl', 'timestamp': ''}, 'inventory': {'url': 'https://pds-rings.seti.org/holdings/metadata/COISS_2xxx/COISS_2999/COISS_2999_inventory.lbl', 'timestamp': ''}, 'moon_summary': {'url': 'https://pds-rings.seti.org/holdings/metadata/COISS_2xxx/COISS_2999/COISS_2999_moon_summary.lbl', 'timestamp': '2019-06-08T16:28:22'}, 'ring_summary': {'url': 'https://pds-rings.seti.org/holdings/metadata/COISS_2xxx/COISS_2999/COISS_2999_ring_summary.lbl', 'timestamp': '2019-06-08T16:29:20'}, 'saturn_summary': {'url': 'https://pds-rings.seti.org/holdings/metadata/COISS_2xxx/COISS_2999/COISS_2999_saturn_summary.lbl', 'timestamp': '2019-06-08T16:30:12'}}}, 'uvis': {'indexes': {'index': {'url': 'https://pds-rings.seti.org/holdings/metadata/COUVIS_0xxx/COUVIS_0999/COUVIS_0999_index.lbl', 'timestamp': '2021-01-20T19:57:23'}, 'moon_summary': {'url': 'https://pds-rings.seti.or

In [ ]:
for mission in missions.keys():
    mdict = missions[mission]
    for instr in mdict.keys():
        instrdict = mdict[instr]
        for index in instrdict['indexes']:
            instrdict['indexes'][index]['timestamp']='test'

In [ ]:
missions

YAML({'cassini': {'iss': {'indexes': {'index': {'url': 'https://pds-rings.seti.org/holdings/metadata/COISS_2xxx/COISS_2999/COISS_2999_index.lbl', 'timestamp': 'test'}, 'inventory': {'url': 'https://pds-rings.seti.org/holdings/metadata/COISS_2xxx/COISS_2999/COISS_2999_inventory.lbl', 'timestamp': 'test'}, 'moon_summary': {'url': 'https://pds-rings.seti.org/holdings/metadata/COISS_2xxx/COISS_2999/COISS_2999_moon_summary.lbl', 'timestamp': 'test'}, 'ring_summary': {'url': 'https://pds-rings.seti.org/holdings/metadata/COISS_2xxx/COISS_2999/COISS_2999_ring_summary.lbl', 'timestamp': 'test'}, 'saturn_summary': {'url': 'https://pds-rings.seti.org/holdings/metadata/COISS_2xxx/COISS_2999/COISS_2999_saturn_summary.lbl', 'timestamp': 'test'}}}, 'uvis': {'indexes': {'index': {'url': 'https://pds-rings.seti.org/holdings/metadata/COUVIS_0xxx/COUVIS_0999/COUVIS_0999_index.lbl', 'timestamp': 'test'}, 'moon_summary': {'url': 'https://pds-rings.seti.org/holdings/metadata/COUVIS_0xxx/COUVIS_0999/COUVIS_0

## The Config() object

The `config` module instantiates a `config` object from the Config class.
Its attributes can be used to access several aspects relevant to the configuration of `planetarypy`.
Using an object approach enables easy growth and nesting over time.

> Note: Any cell that starts with an `# export` becomes part of the library.
  Any other cells become automatically part of tests.
  `exports` also adds the exported code to the docs.
  
> Note: A good "First issue" for collaborators would be to improve the Config class to be able to merge a new larger index catalog with an existing one the user might have configured. This will need to take care of existing timestamps and possible other indexes that were put in by the user.

In [ ]:
for mission in config.list_missions():
    for instrument in config.list_instruments(mission):
        for index in config.list_indexes(f"{mission}.{instrument}"):
#             key = ".".join(['missions', mission, instrument, index])
            print(index)

index
inventory
moon_summary
ring_summary
saturn_summary
index
moon_summary
ring_summary
saturn_summary
supplemental_index
versions
dtm
edr
rdr
edr
edr1
edr2
rdr1
rdr2
edr
rdr


In [ ]:
config = Config(config_path='test_config.yaml')

In [ ]:
# export
config = Config()

In [ ]:
config.storage_root

Path('/home/maye/big_drive/planetary_data')

In [ ]:
show_doc(Config.read_config)

<h4 id="Config.read_config" class="doc_header"><code>Config.read_config</code><a href="__main__.py#L28" class="source_link" style="float:right">[source]</a></h4>

> <code>Config.read_config</code>()

```
Read the configfile and store config dict.

If found, load config via `yaml` and store YAML dict as `d`.
`storage_root` will be stored as attribute.
```

In [ ]:
assert isinstance(config.yamldic, yaml.YAML)

In [ ]:
assert isinstance(config.d, dict)

In [ ]:
show_doc(Config.get_value)

<h4 id="Config.get_value" class="doc_header"><code>Config.get_value</code><a href="__main__.py#L45" class="source_link" style="float:right">[source]</a></h4>

> <code>Config.get_value</code>(**`key`**)

```
Get sub-dictionary by nested key.

Parameters
----------
nested_key: str
    A nested key in dotted format, e.g. cassini.uvis.indexes
```

In [ ]:
index = 'missions.cassini.iss.indexes.moon_summary'

In [ ]:
config.get_value(index)

{'url': 'https://pds-rings.seti.org/holdings/metadata/COISS_2xxx/COISS_2999/COISS_2999_moon_summary.lbl',
 'timestamp': '2019-06-08T16:28:22'}

In [ ]:
show_doc(Config.set_value)

<h4 id="Config.set_value" class="doc_header"><code>Config.set_value</code><a href="__main__.py#L55" class="source_link" style="float:right">[source]</a></h4>

> <code>Config.set_value</code>(**`nested_key`**, **`value`**)

```
Set sub-dic using dotted key.

Parameters
----------
key: str
    A nested key in dotted format, e.g. cassini.uvis.ring_summary
value: convertable to string
    Value for the given key to be stored.
```

In [ ]:
show_doc(Config.save)

<h4 id="Config.save" class="doc_header"><code>Config.save</code><a href="__main__.py#L72" class="source_link" style="float:right">[source]</a></h4>

> <code>Config.save</code>()

```
Write the YAML dict to file.
```

In [ ]:
config.get_value(index)

{'url': 'https://pds-rings.seti.org/holdings/metadata/COISS_2xxx/COISS_2999/COISS_2999_moon_summary.lbl',
 'timestamp': '2019-06-08T16:28:22'}

In [ ]:
show_doc(Config.ask_storage_root)

<h4 id="Config.ask_storage_root" class="doc_header"><code>Config.ask_storage_root</code><a href="__main__.py#L79" class="source_link" style="float:right">[source]</a></h4>

> <code>Config.ask_storage_root</code>()

```
Use input() to ask user for the storage_root path.

The path will be stored in the YAML-dict and saved into existing config file
at `Class.path`, either default or as given during init.
`storage_root` attribute is set as well.
```

In [ ]:
config.list_missions()

['cassini', 'mro', 'lro']

In [ ]:
config.fname

'planetarypy_config.yaml'

In [ ]:
config.path

Path('/home/maye/.planetarypy_config.yaml')

In [ ]:
config.list_instruments('cassini')

['iss', 'uvis']

In [ ]:
config.list_indexes('cassini.iss')

['index', 'inventory', 'moon_summary', 'ring_summary', 'saturn_summary']

In [ ]:
config.d

{'storage_root': '/home/maye/big_drive/planetary_data',
 'missions': {'cassini': {'iss': {'indexes': {'index': {'url': 'https://pds-rings.seti.org/holdings/metadata/COISS_2xxx/COISS_2999/COISS_2999_index.lbl',
      'timestamp': ''},
     'inventory': {'url': 'https://pds-rings.seti.org/holdings/metadata/COISS_2xxx/COISS_2999/COISS_2999_inventory.lbl',
      'timestamp': ''},
     'moon_summary': {'url': 'https://pds-rings.seti.org/holdings/metadata/COISS_2xxx/COISS_2999/COISS_2999_moon_summary.lbl',
      'timestamp': '2019-06-08T16:28:22'},
     'ring_summary': {'url': 'https://pds-rings.seti.org/holdings/metadata/COISS_2xxx/COISS_2999/COISS_2999_ring_summary.lbl',
      'timestamp': '2019-06-08T16:29:20'},
     'saturn_summary': {'url': 'https://pds-rings.seti.org/holdings/metadata/COISS_2xxx/COISS_2999/COISS_2999_saturn_summary.lbl',
      'timestamp': '2019-06-08T16:30:12'}}},
   'uvis': {'indexes': {'index': {'url': 'https://pds-rings.seti.org/holdings/metadata/COUVIS_0xxx/COUVIS

In [ ]:
for mission in config.list_missions():
    for instrument in config.list_instruments(mission):
        for index in toclean['missions'][mission][instrument]['indexes'].keys():
            key = ".".join(['missions', mission, instrument, index])
            print(key)

<class 'strictyaml.representation.YAML'>


TypeError: sequence item 1: expected str instance, YAML found

In [ ]:
config.list_missions()

['cassini', 'mro', 'lro']

In [ ]:
config = Config(config_path='test_config.yaml')